In [1]:
import sympy
import pandas as pd
from tqdm import tqdm
from symcircuit.symbolic_solver.domains import ExperimentResult

# Loading the Results of VLSI CMMF Exploration

In [2]:
results = ExperimentResult("VLSI_CMMF_POST_EXPERIMENT")
dir_of_results = results.find_results_file("Runs/VLSI_CMMF_Automated_NA")
results = results.load(f"{dir_of_results[0]}/results.pkl")

In [ ]:
results.classifications_dict.keys()

In [ ]:
classifications_of_intrest = results.classifications_dict["Z1_Z2_Z3_Z4_Z5_Z6"]
len(classifications_of_intrest), classifications_of_intrest[0]

In [5]:
# df = results.flatten_classifications() # This way we can access all the classsifcation into a pandas df

In [ ]:
classifications_of_intrest = []

for key in results.classifications_dict.keys():
    classifications = results.classifications_dict[key]

    classifications_of_intrest += ([classification 
                                for classification in classifications 
                                if (classification.fType in ["BP"] )]
                                )
    
    fTypes = set(classification.fType for classification in classifications_of_intrest)

len(classifications_of_intrest), fTypes

In [ ]:
idx = 3
classifications_of_intrest[idx].zCombo

In [ ]:
classifications_of_intrest[idx].transferFunc

In [ ]:
Q = classifications_of_intrest[idx].parameters["Q"].simplify()
Q

In [ ]:
K = classifications_of_intrest[idx].parameters["K_BP"].simplify()
K

In [ ]:
wo = classifications_of_intrest[idx].parameters["wo"].simplify()
wo

In [ ]:
k_var = [var for var in K.free_symbols]
k_var

In [ ]:
sub_dict = {
    k_var[3] : sympy.symbols("R"),
    k_var[0] : sympy.symbols("R"),
    k_var[1] : sympy.symbols("R"),
    k_var[-1] : sympy.symbols("R"), 
    k_var[-2] : sympy.symbols("R"),
}
sub_dict

In [ ]:
sub_dict = {
    k_var[2] : sympy.symbols("C"),
    k_var[4] : sympy.symbols("C"),

}
sub_dict

In [ ]:
K.subs(sub_dict).simplify()

In [ ]:
Q.subs(sub_dict).simplify()

In [ ]:
wo.subs(sub_dict).simplify()

In [ ]:
sym_dict = [sym for sym in classifications_of_intrest[idx].transferFunc.free_symbols]
sym_dict

In [ ]:
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np

# Define the design variables
R1, C1, s, f = sp.symbols('R1 C1 s f')

# Define your transfer function (example: a simple RC low-pass filter)
H = classifications_of_intrest[idx].transferFunc

# Substitute design variable values (example)
R1_val = 1e3  # Ohms
R2_val = 1e3  # Ohms
R3_val = 1e3  # Ohms
R4_val = 1e3  # Ohms
R5_val = 1e3  # Ohms
R6_val = 1e3  # Ohms


C4_val = 2e-9    # Farads
C5_val = C4_val  # Farads
# C_4, R_1, R_2, R_6, s, R_5, C_5, R_3
# Define the transfer function in terms of frequency
H_numeric = H.subs({
    sym_dict[0]: C4_val, 
    sym_dict[1] : R1_val, 
    sym_dict[2] : R2_val,
    sym_dict[3] : R6_val,
    sym_dict[5] : R5_val,
    sym_dict[6] : C5_val,
    sym_dict[7] : R3_val,
    s: 2 * sp.pi * sp.I * f})

# Compute the magnitude (in dB) and phase (in degrees)
magnitude_expr = 20 * sp.log(abs(H_numeric), 10)  # Magnitude in dB
phase_expr = sp.arg(H_numeric) * 180 / sp.pi   # Phase in degrees

# Define the frequency range
frequencies = np.logspace(1, 7, 100)  # From 1 Hz to 1 MHz

# Evaluate the magnitude and phase for each frequency
magnitude_vals = [magnitude_expr.subs(f, freq).evalf() for freq in frequencies]
phase_vals = [phase_expr.subs(f, freq).evalf() for freq in frequencies]

# Plotting
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Plot Magnitude
axs[0].semilogx(frequencies, [float(m) for m in magnitude_vals], label='Magnitude')
axs[0].set_title("Bode Plot - Magnitude")
axs[0].set_xlabel("Frequency (Hz)")
axs[0].set_ylabel("Magnitude (dB)")
axs[0].grid(True)  # Add grid
axs[0].legend()

# Plot Phase
axs[1].semilogx(frequencies, [float(p) for p in phase_vals], label='Phase')
axs[1].set_title("Bode Plot - Phase")
axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_ylabel("Phase (degrees)")
axs[1].grid(True)  # Add grid
axs[1].legend()

# Show the plot with interactive features
plt.tight_layout()
plt.show()


In [ ]:
classifications_of_intrest[idx].zCombo

In [28]:
gb = sympy.symbols("GB")

# Examining the Higher Order Filters 

In [ ]:
classifications_of_intrest = []

for key in results.classifications_dict.keys():
    classifications = results.classifications_dict[key]

    classifications_of_intrest += ([classification 
                                for classification in classifications 
                                if (classification.fType in ["X-INVALID-ORDER", "X-INVALID-NUMER"] )]
                                )
    
    fTypes = set(classification.fType for classification in classifications_of_intrest)

len(classifications_of_intrest), fTypes

## Update filterOrders

In [ ]:
count = {
    "X-CONST" : 0,
}
s = sympy.symbols("s")

for i, classification in enumerate(classifications_of_intrest):
    tf = classification.transferFunc

    if not (s in tf.free_symbols): # TF id NOT a function of S - Pure amplifier/level-shifter
        ftype = "X-CONST"
        count[ftype] += 1
        classifications_of_intrest[i].fType = "X-CONST"
        classifications_of_intrest[i].filterOrder = "Zero"

count

## Load the old classification into a classifier object

In [ ]:
from symcircuit.symbolic_solver.filter import Filter_Classifier

classifier = Filter_Classifier()
classifier.overwrite_classifications(classifications_of_intrest)
classifier.isClassified(), len(classifier.classifications)


In [21]:
# classifier.classifyFilter(filterOrder="FirstOrder")

In [22]:
# classifier.summarizeFilterType()

# Analyze higher orders
- **LP-3rd order**: num_deg = 0, denom_deg = 3
    -  The denom is decomposable into 1 second-orders and 1 first-order
- **HP-4th Order**: num_deg = 4, denom_deg = 4
    -  N(s) has only s^4 term
- **BP-4th order**: num_deg = 0, denom_deg = 4
    -  N(s) only has s^2 term
    -  The denom is decomposable into 2 second-orders

### LP-3rd Order

In [ ]:
from tqdm import tqdm
# from tqdm.gui import tqdm

def get_3rd_order_lp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=3, numer_order=0, printMessage=False)
    print(f"{count} candidates for 3rd-order LP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not classifier.is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not classifier.is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break
        gain = None
        if valid:
            count_valid += 1
            tf = classification.transferFunc
            gain = sympy.limit(tf, sympy.symbols("s"), 0)
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "tf" : sympy.latex(classification.transferFunc),
            "DC-Gain": gain,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output
output_lp = get_3rd_order_lp(classifier)

df_lp = pd.DataFrame(output_lp)
df_lp.to_csv(f"Runs/{results.experiment_name}/3rd_LP.csv")
df_lp

In [ ]:
filtered = [output for output in output_lp if output["valid"]]
idx = 25
x = filtered[idx] 
x["DC-Gain"]

In [ ]:
x["denom"][1]

In [ ]:
x["denom"][0]

In [ ]:
x["zCombo"]

### HP-4th Order

In [ ]:

def get_4th_order_hp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=4, printMessage=False)
    print(f"{count} candidates for 4th-order HP")

    output = []
    count = 0
    count_valid = 0 
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False
        # else:
        #     for exp in numer[0].as_dict():
        #         if exp != 4:
        #             valid = False


        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_hp = get_4th_order_hp(classifier)

df_hp = pd.DataFrame(output_hp)
df_hp.to_csv(f"Runs/{results.experiment_name}/4th_HP.csv")
df_hp

### BP-4th Order

In [ ]:
classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)

def get_4th_order_bp(classifier, check_for_stability: bool = False):
    classifications, count = classifier.findFilterInClassification(denom_order=4, numer_order=2, printMessage=False)
    print(f"{count} candidates for 4th-order BP")

    output = []
    count = 0
    count_valid = 0
    for classification in tqdm(classifications, total=len(classifications)):
        count += 1
        tf = classification.transferFunc
        k, numer, denom = classifier.decompose_tf(tf)

        valid = True
        for poly in denom:
            order = poly.degree()
            if order == 1: 
                if (len(poly.as_dict()) != 2) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break
                
            elif order == 2:
                if (len(poly.as_dict()) != 3) or (check_for_stability and not is_poly_stable(poly)):
                    valid = False
                    break

            else:
                valid = False
                break

        if len(numer) != 1:
            valid = False

        if valid:
            count_valid += 1
            # print(f"ID {count} - valid")

        output += [{
            "valid": valid,
            "zCombo": classification.zCombo,
            "classification" : classification,
            "k" :  k,
            "numer": numer,
            "denom": denom,
            "num-factor-count":len(numer),
            "denom-factor-count": len(denom)
        }]

    print(f"{count_valid} verified filters")

    return output

output_bp = get_4th_order_bp(classifier)

df_bp = pd.DataFrame(output_bp)
df_bp.to_csv(f"Runs/{results.experiment_name}/4th_BP.csv")
df_bp

# Extract information from all

In [ ]:
data = []
for x in tqdm(classifier.classifications, total = len(classifier.classifications)):
    k, numer, denom = classifier.decompose_tf(x.transferFunc)
    
    degrees_list_denom = [poly.degree() for poly in denom]
    if len(degrees_list_denom) != 0:
        max_degrees_denom = max(degrees_list_denom)
    else: 
        max_degrees_denom = -1

    degrees_list_numer = [poly.degree() for poly in numer]

    if len(degrees_list_numer) != 0:
        max_degrees_numer = max(degrees_list_numer)
    else: 
        max_degrees_numer = -1

    data += [
        {
            "combo" : x.zCombo,
            "TF" : sympy.latex(x.transferFunc),
            "param" : x.parameters,
            "type" : x.fType,

            "denom order" : x.tf_denom_order,
            "numer order" : x.tf_numer_order,

            "numer facator count" : len(numer),
            "denom factor count" : len(denom),

            "numer factor degrees": degrees_list_numer,
            "denom factor degrees": degrees_list_denom,

            "max denom degree" : max_degrees_denom,
            "max numer degree" : max_degrees_numer,

            "k" : k,
            "numer factors" : numer,
            "denom factors" :denom

        }
    ]

df = pd.DataFrame(data)
df.to_csv(f"Runs/{results.experiment_name}/consice_summary.csv")

# Playground

In [ ]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function, variable, assumptions={}):
    """
    Decomposes the denominator of a transfer function into factors, 
    grouping strictly by the specified variable (e.g., `s`).

    Args:
        transfer_function: A Sympy expression representing the transfer function.
        variable: The main variable to group by (e.g., `s`).
        assumptions: A dictionary of assumptions for symbolic variables.

    Returns:
        A dictionary where each key is a factor (grouped by the main variable),
        and the value contains its roots and properties.
    """
    # Apply assumptions to all symbols
    for var, assumption in assumptions.items():
        var._assumptions.update({assumption: True})
    
    # Extract the denominator
    denominator = transfer_function.as_numer_denom()[1]
    
    # Ensure the denominator is treated as a polynomial in the given variable
    poly = Poly(denominator, variable)
    
    # Factorize strictly with respect to `variable`
    factored = factor(poly.as_expr())
    
    # Inspect each factor
    factors = factored.as_ordered_factors()
    results = {}
    for factor_expr in factors:
        # Solve for the roots of this factor
        roots = solveset(factor_expr, variable, domain=S.Complexes)
        
        # Analyze each root (e.g., real part)
        roots_info = []
        for root in roots:
            real_part = simplify(root.as_real_imag()[0])
            roots_info.append({
                "root": root,
                "real_part": real_part
            })
        
        # Store results for this factor
        results[str(factor_expr)] = {
            "factor": factor_expr,
            "roots": roots_info,
            "degree": Poly(factor_expr, variable).degree()
        }
    
    return results

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (L1 * s**3 + (R2 + R3) * s**2 + 1 / C1 * s + 1 / C2)

# Decompose and inspect the factors
results = decompose_tf(G, s)
for factor, info in results.items():
    print(f"Factor: {info['factor']}")
    print(f"Degree: {info['degree']}")
    print("Roots:")
    for root_info in info["roots"]:
        print(f"  - Root: {root_info['root']}, Real part: {root_info['real_part']}")
    print()


In [ ]:
results.keys()

In [51]:
from sympy import symbols, Poly, factor, simplify, solveset, S

def decompose_tf(transfer_function):
    s = symbols("s")

    # Extract the numerator and denominator
    numerator, denominator = transfer_function.as_numer_denom()
 
    numerator = Poly(numerator, s)
    denominator = Poly(denominator, s)

    # Factorize strictly with respect to `variable`
    numerator = factor(numerator.as_expr())
    denominator = factor(denominator.as_expr())

    # Inspect each factor
    factors_numer = numerator.as_ordered_factors()
    factors_denom = denominator.as_ordered_factors()
    
    return factors_numer, factors_denom

# Example: Define symbolic components
R1, R2, R3 = symbols('R1 R2 R3', positive=True)  # Resistors
L1 = symbols('L1', positive=True)  # Inductor
C1, C2 = symbols('C1 C2', positive=True)  # Capacitors
s = symbols('s')

# Example transfer function
G = (s + R1) / (((s+R3)*(s**2 + s*2*C2+ 2*C2**2)).expand())
G.expand()

# Decompose and inspect the factors
tup = decompose_tf(G)
numer = tup[0]
denom = tup[1]
